In [1]:
import wikipedia as wp
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas.io.html import read_html

# Reads in the Data Set

**Number of Guns per Capita Data Set**

In [2]:
html = wp.page("Estimated number of civilian guns per capita by country").html().encode("UTF-8")
gunsPerCapita = pd.read_html(html)[0]

**Homocide Rate per Country Data Set**

In [3]:
page = "https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate"
murderRate = pd.read_html(page, attrs={"class":"wikitable sortable"})[1]

**Nominal GDP per Country Data Set**

In [4]:
html3 = wp.page("List of countries by GDP (nominal)").html().encode("UTF-8")
GDP = pd.read_html(html3)[3]

# Cleaning Data Sets

**Number of Guns per Capita Data Set Cleaning**